# CARGA DE LIBRERÍAS PARA PROCESO ETL

Paso 1: Extracción (Extracción de datos del archivo CSV)

Primero, necesitas importar la biblioteca pandas y cargar el archivo CSV en un DataFrame.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import ast
import pyarrow as pa
import pyarrow.parquet as pq

# PRIMER DATAFRAME: 'movies_dataset.csv'

In [2]:
# Se Carga el archivo CSV en un DataFrame
movies = pd.read_csv('..\movies_dataset.csv', encoding='utf-8')

In [3]:
movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [4]:
movies.info() #Se revisa el tipo de datos de cada una de las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Paso 2: Transformación (Limpieza y preparación de los datos)

Aquí, puedes realizar operaciones de limpieza y preparación de los datos, como eliminar filas duplicadas, manejar valores faltantes, convertir tipos de datos, etc.

In [5]:
movies_etl = movies #Se crea una copia del DataFrame original para realizar las limpiezas

In [6]:
# Columnas a eliminar pedidas por el cliente
movies_etl = movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [7]:
# Los valores nulos del campo release_date deben eliminarse.
movies_etl = movies_etl.dropna(subset=['release_date'])

In [8]:
# Los valores nulos del campo runtime deben eliminarse.
movies_etl = movies_etl.dropna(subset=['runtime'])

In [9]:
# Los valores nulos del campo status deben eliminarse.
movies_etl = movies_etl.dropna(subset=['status'])

In [10]:
# La función isnull() identifica los valores faltantes y la función sum() los cuenta. Luego se muestra el resultado.
faltantes_por_columna = movies_etl.isnull().sum()
faltantes_por_columna

belongs_to_collection    40575
budget                       0
genres                       0
id                           0
original_language           11
overview                   683
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                      0
spoken_languages             0
status                       0
tagline                  24669
title                        0
vote_average                 0
vote_count                   0
dtype: int64

In [11]:
# Los valores nulos del campo title deben eliminarse.
movies_etl = movies_etl.dropna(subset=['title'])

In [12]:
# Los valores nulos del campo vote_average deben eliminarse.
movies_etl = movies_etl.dropna(subset=['vote_average'])

In [13]:
# Los valores nulos del campo vote_count deben eliminarse.
movies_etl = movies_etl.dropna(subset=['vote_count'])

In [14]:
# Se inserta el campo 'id' como primer columna del DataFrame 
movies_etl.insert(0, 'id', movies_etl.pop('id')) 

### Cambios de tipo de dato en las columnas

In [15]:
movies_etl['id'] = movies_etl['id'].astype(int) # Convertir la columna 'id' a tipo entero.
movies_etl['popularity'] = movies_etl['popularity'].astype(float) # Convertir la columna 'popularity' a tipo flotante, este dato sirve como ranking debido a una opinión de TMBD.
movies_etl['runtime'] = movies_etl ['runtime'].fillna(0).astype(int) # Se convierte a entero ya que esta columna es la duración de la película, y sirve para posteriores operaciones. 
movies_etl['vote_count'] = movies_etl['vote_count'].fillna(0).astype(int) # Se convierte a entero ya que esta columna es el número de votos, y sirve para hacer algún cálculo posterior.
movies_etl['release_date'] = pd.to_datetime(movies_etl['release_date'] , errors='coerce') # Convertir la columna release_date a formato de fecha
movies_etl['title'] = movies_etl['title'].astype(str) # Convertir la columna title a tipo string.
movies_etl['overview'] = movies_etl['overview'].astype(str) # Convertir la columna overview a tipo string, esta es la descripción de la película. 
movies_etl['tagline'] = movies_etl['tagline'].astype(str) # Str ya que es texto
movies_etl['original_language'] = movies_etl['original_language'].astype(str) # Str ya que es texto
movies_etl['status'] = movies_etl['status'].astype(str) # Str ya que es texto
movies_etl['budget'] = movies_etl['budget'].astype(int) # Convertir la columna budget a tipo entero, ya que es el presupuesto de la película y sirve para posteriores operaciones.
movies_etl['revenue'] = movies_etl['revenue'].astype(int) # Convertir la columna revenue a tipo entero, ya que es el ingreso de la película por la venta de entradas y sirve para posteriores operaciones.

In [16]:
movies_etl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45054 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45054 non-null  int64         
 1   belongs_to_collection  4479 non-null   object        
 2   budget                 45054 non-null  int64         
 3   genres                 45054 non-null  object        
 4   original_language      45054 non-null  object        
 5   overview               45054 non-null  object        
 6   popularity             45054 non-null  float64       
 7   production_companies   45054 non-null  object        
 8   production_countries   45054 non-null  object        
 9   release_date           45054 non-null  datetime64[ns]
 10  revenue                45054 non-null  int64         
 11  runtime                45054 non-null  int64         
 12  spoken_languages       45054 non-null  object        
 13  status

In [17]:
# Extraer el año de la fecha de estreno y crear una nueva columna release_year
movies_etl['release_year'] = movies_etl['release_date'].dt.year

In [18]:
# Convertir la columna release_year a tipo entero
movies_etl['release_year'] = movies_etl['release_year'].astype(int)

In [19]:
movies_etl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45054 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45054 non-null  int64         
 1   belongs_to_collection  4479 non-null   object        
 2   budget                 45054 non-null  int64         
 3   genres                 45054 non-null  object        
 4   original_language      45054 non-null  object        
 5   overview               45054 non-null  object        
 6   popularity             45054 non-null  float64       
 7   production_companies   45054 non-null  object        
 8   production_countries   45054 non-null  object        
 9   release_date           45054 non-null  datetime64[ns]
 10  revenue                45054 non-null  int64         
 11  runtime                45054 non-null  int64         
 12  spoken_languages       45054 non-null  object        
 13  status

In [20]:
# Esta línea de código utiliza la función np.where para evaluar la condición movies_etl['budget'] != 0, si la condición es verdadera, calcula el retorno de la inversión, de lo contrario, asigna el valor cero.
movies_etl['return'] = np.where(movies_etl['budget'] != 0, movies_etl['revenue'] / movies_etl['budget'], 0)

In [21]:
# Contabilizar las filas duplicadas y mostrar el resultado en la consola.
duplicados = movies_etl[movies_etl.duplicated(keep=False)]
cantidad_duplicados = duplicados.shape[0]
print(f"Cantidad de filas duplicadas: {cantidad_duplicados}")

Cantidad de filas duplicadas: 33


In [22]:
# Elimina filas duplicadas
movies_etl = movies_etl.drop_duplicates()

In [23]:
# Contabilizar las filas duplicadas y mostrar el resultado en la consola.
duplicados = movies_etl[movies_etl.duplicated(keep=False)]
cantidad_duplicados = duplicados.shape[0]
print(f"Cantidad de filas duplicadas: {cantidad_duplicados}")

Cantidad de filas duplicadas: 0


In [24]:
movies_etl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45037 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45037 non-null  int64         
 1   belongs_to_collection  4478 non-null   object        
 2   budget                 45037 non-null  int64         
 3   genres                 45037 non-null  object        
 4   original_language      45037 non-null  object        
 5   overview               45037 non-null  object        
 6   popularity             45037 non-null  float64       
 7   production_companies   45037 non-null  object        
 8   production_countries   45037 non-null  object        
 9   release_date           45037 non-null  datetime64[ns]
 10  revenue                45037 non-null  int64         
 11  runtime                45037 non-null  int64         
 12  spoken_languages       45037 non-null  object        
 13  status

In [25]:
# Contabilizar los campos faltantes por columna
faltantes_por_columna = movies_etl.isnull().sum()
faltantes_por_columna

id                           0
belongs_to_collection    40559
budget                       0
genres                       0
original_language            0
overview                     0
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                      0
spoken_languages             0
status                       0
tagline                      0
title                        0
vote_average                 0
vote_count                   0
release_year                 0
return                       0
dtype: int64

Desanidado de la columna 'Belongs_to_collection'

In [26]:
# Declaramos una lista vacia que contendra los datos limpios
collection_name = []

# Iteramos todas las filas
for index, row in movies_etl.iterrows():
    string = str(row['belongs_to_collection']).strip('{}') # Removemos las llaves
    lista = string.split(',') # Almacenamos cada elemento del string
    
# Condicional para manejar celdas vacias
    if len(lista) >= 2: 
        collection_name.append(str(lista[1]).replace(" 'name': ", "").strip('[""]').strip("''")) # Almacena y limpia el nombre de la coleccion

# Si la celda esta vacia se debe almacenar un elemento vacio para conservar la estructura de datos    
    else:
        collection_name.append('')

In [27]:
# Reemplazamos los datos de la columna por los de la lista

movies_etl['belongs_to_collection'] = collection_name

Desanidado de la columna 'genre'

In [28]:
# Declaramos lista vacia para nombres de género
all_genre_names = []

# Recorremos cada fila del DataFrame
for index, row in movies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        genre_list = ast.literal_eval(str(row['genres']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        genre_list = []

    # Lista temporal para nombres de género en la fila actual
    genre_names = []

    # Extraemos los nombres de cada diccionario en la lista temporal
    for genre in genre_list:
        genre_names.append(genre.get('name', ''))

    # Añadimos las listas temporales a las listas finales
    all_genre_names.append(genre_names)

# Revisamos el resultado
all_genre_names

[['Animation', 'Comedy', 'Family'],
 ['Adventure', 'Fantasy', 'Family'],
 ['Romance', 'Comedy'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy'],
 ['Action', 'Crime', 'Drama', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Action', 'Adventure', 'Drama', 'Family'],
 ['Action', 'Adventure', 'Thriller'],
 ['Adventure', 'Action', 'Thriller'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Horror'],
 ['Family', 'Animation', 'Adventure'],
 ['History', 'Drama'],
 ['Action', 'Adventure'],
 ['Drama', 'Crime'],
 ['Drama', 'Romance'],
 ['Crime', 'Comedy'],
 ['Crime', 'Comedy', 'Adventure'],
 ['Action', 'Comedy', 'Crime'],
 ['Comedy', 'Thriller', 'Crime'],
 ['Drama', 'Thriller'],
 ['Action', 'Adventure', 'Crime', 'Thriller'],
 ['Drama', 'Fantasy', 'Science Fiction', 'Thriller'],
 ['Drama', 'Romance'],
 ['Drama'],
 ['Comedy', 'Drama', 'Family'],
 ['Drama', 'Romance'],
 ['Fantasy', 'Science Fiction', 'Adventure'],
 ['Drama', 'Crime'],
 ['Drama', 'Crime'],
 ['Science Fiction', 'Thriller', 'Mystery'],
 ['Romance

In [29]:
# Reemplazamos los datos de la columna por los del diccionario

movies_etl['genres'] = all_genre_names

Desanidado de la columna 'production_companies'

In [30]:
# Declaramos lista vacia para nombres de companias
all_companies_names = []

# Recorremos cada fila del DataFrame
for index, row in movies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        companies_list = ast.literal_eval(str(row['production_companies']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        companies_list = []

    # Lista temporal para nombres de companias en la fila actual
    companies_names = []

    # Extraemos los nombres de cada diccionario en la lista temporal
    for companies in companies_list:
        companies_names.append(companies.get('name', ''))

    # Añadimos las listas temporales a las listas finales
    all_companies_names.append(companies_names)

# Revisamos el resultado
all_companies_names

[['Pixar Animation Studios'],
 ['TriStar Pictures', 'Teitler Film', 'Interscope Communications'],
 ['Warner Bros.', 'Lancaster Gate'],
 ['Twentieth Century Fox Film Corporation'],
 ['Sandollar Productions', 'Touchstone Pictures'],
 ['Regency Enterprises', 'Forward Pass', 'Warner Bros.'],
 ['Paramount Pictures',
  'Scott Rudin Productions',
  'Mirage Enterprises',
  'Sandollar Productions',
  'Constellation Entertainment',
  'Worldwide',
  'Mont Blanc Entertainment GmbH'],
 ['Walt Disney Pictures'],
 ['Universal Pictures', 'Imperial Entertainment', 'Signature Entertainment'],
 ['United Artists', 'Eon Productions'],
 ['Columbia Pictures', 'Castle Rock Entertainment'],
 ['Columbia Pictures', 'Castle Rock Entertainment', 'Enigma Pictures'],
 ['Universal Pictures', 'Amblin Entertainment', 'Amblimation'],
 ['Hollywood Pictures', 'Cinergi Pictures Entertainment'],
 ['Le Studio Canal+',
  'Laurence Mark Productions',
  'Metro-Goldwyn-Mayer (MGM)',
  'Carolco Pictures'],
 ['Universal Pictures',

In [31]:
# Reemplazamos los datos de la columna por los del diccionario

movies_etl['production_companies'] = all_companies_names

Desanidado de la columna 'production_countries'

In [32]:
# Declaramos lista vacia para nombres de paises
all_countries_names = []

# Recorremos cada fila del DataFrame
for index, row in movies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        countries_list = ast.literal_eval(str(row['production_countries']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        countries_list = []

    # Lista temporal para nombres de paises en la fila actual
    countries_names = []

    # Extraer los datos de cada diccionario en la lista
    for countries in countries_list:
        countries_names.append(countries.get('name', ''))  # Añadir el nombre

    # Añadir las listas temporales a las listas finales
    all_countries_names.append(countries_names)

# Revisamos el resultado
all_countries_names

[['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['Germany', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United Kingdom', 'United States of America'],
 ['United States of America'],
 ['France', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['France', 'Germany', 'Italy', 'United States of America'],
 ['France', 'United States of America'],
 ['United Kingdom', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['France', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['Italy'],
 ['United States of America'],
 ['United Kingdom', 'France'],
 ['France', 'Germany', 'Spain'],
 ['China', 

In [33]:
# Reemplazamos los datos de la columna por los del diccionario

movies_etl['production_countries'] = all_countries_names

Desanidado de la columna 'spoken_languages'

In [34]:
# Declaramos lista vacia para lenguajes
all_languages_names = []

# Recorremos cada fila del DataFrame
for index, row in movies_etl.iterrows():
    # Convertimos strings a lista de diccionarios
    try:
        languages_list = ast.literal_eval(str(row['spoken_languages']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        languages_list = []

    # Lista temporal para lenguajes en la fila actual
    languages_names = []

    # Extraer los datos de cada diccionario en la lista
    for languages in languages_list:
        languages_names.append(languages.get('name', ''))  # Añadir el nombre

    # Añadir las listas temporales a las listas finales
    all_languages_names.append(languages_names)

# Revisamos el resultado
all_languages_names

[['English'],
 ['English', 'Français'],
 ['English'],
 ['English'],
 ['English'],
 ['English', 'Español'],
 ['Français', 'English'],
 ['English', 'Deutsch'],
 ['English'],
 ['English', 'Pусский', 'Español'],
 ['English'],
 ['English', 'Deutsch'],
 ['English'],
 ['English'],
 ['English', 'Latin'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', 'Deutsch'],
 ['English', 'Español', 'Nederlands'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['广州话 / 廣州話', 'Français'],
 ['普通话'],
 ['English'],
 ['English', 'Français'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['Pусский', 'English'],
 ['English'],
 ['Español', 'English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', 'Español'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['Español', 'English', 'Français', 'Magyar'],
 [],
 ['English'],
 ['shqip', 'Italiano'],
 ['English', 'Deutsch'],
 ['English'],
 ['Eng

In [35]:
# Reemplazamos los datos de la columna por los del diccionario

movies_etl['spoken_languages'] = all_languages_names

In [36]:
movies_etl

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,862,Toy Story Collection,30000000,"[Animation, Comedy, Family]",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033,81,[English],Released,nan,Toy Story,7.7,5415,1995,12.451801
1,8844,,65000000,"[Adventure, Fantasy, Family]",en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249,104,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413,1995,4.043035
2,15602,Grumpy Old Men Collection,0,"[Romance, Comedy]",en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0,101,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92,1995,0.000000
3,31357,,16000000,"[Comedy, Drama, Romance]",en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156,127,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34,1995,5.090760
4,11862,Father of the Bride Collection,0,[Comedy],en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911,106,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,30840,,0,"[Drama, Action, Romance]",en,"Yet another version of the classic epic, with ...",5.683753,"[Westdeutscher Rundfunk (WDR), Working Title F...","[Canada, Germany, United Kingdom, United State...",1991-05-13,0,104,[English],Released,nan,Robin Hood,5.7,26,1991,0.000000
45462,111109,,0,[Drama],tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,0,360,[],Released,nan,Century of Birthing,9.0,3,2011,0.000000
45463,67758,,0,"[Action, Drama, Thriller]",en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,0,90,[English],Released,A deadly game of wits.,Betrayal,3.8,6,2003,0.000000
45464,227506,,0,[],en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,0,87,[],Released,nan,Satan Triumphant,0.0,0,1917,0.000000


In [37]:
duplicados_id = movies_etl['id'].duplicated().sum()
print(f"Cantidad de valores duplicados en la columna 'id': {duplicados_id}")

Cantidad de valores duplicados en la columna 'id': 13


In [38]:
filas_duplicadas = movies_etl[movies_etl.duplicated(subset='id', keep=False)]
print(filas_duplicadas)

           id belongs_to_collection    budget  \
838    132641                               0   
949     22649                               4   
2564    84198                               0   
4114    10991    Pokémon Collection  16000000   
4356   110428                         3512454   
5130    15028                        26000000   
5535    12600    Pokémon Collection         0   
5710   109962                               0   
5865     4912                        30000000   
9576    69234                        10000000   
10419   14788                         1600000   
11155   77221                        40000000   
11342   13209                            2500   
12066   14788                         1600000   
15074   22649                               4   
15765   13209                            2500   
20843   77221                        40000000   
20899  109962                               0   
21116   84198                               0   
23534  110428       

In [39]:
# Se establece anteriormente que la columna identificadora del registro es la columna 'id', todos los registros son iguales en la filas, excepto la columna 'popularity'
# Por lo cual para eliminar estos duplicados para efectos del análisis, se decide eliminar los registros con la columna 'popularity' de menor calificación numérica. 
movies_etl = movies_etl.sort_values('popularity', ascending=False).drop_duplicates('id', keep='first')

In [40]:
# Se verifica la cantidad de duplicados después de realizar la eliminación de los duplicados
duplicados_id = movies_etl['id'].duplicated().sum()
print(f"Cantidad de valores duplicados en la columna 'id': {duplicados_id}")

Cantidad de valores duplicados en la columna 'id': 0


In [41]:
# Esta función calcula la diferencia entre cada índice y el anterior, y verifica si todas las diferencias son iguales a 1.
if (movies_etl.index.diff() == 1).all():
    print("Los índices son consecutivos")
else:
    print("Los índices no son consecutivos")

Los índices no son consecutivos


In [42]:
# Verificar si los índices son consecutivos
def chequeo_indices_consecutivos(movies_etl):
    # Obtener los índices como una lista
    index_list = movies_etl.index.tolist()
    # Generar una lista de índices consecutivos esperados
    expected_indices = list(range(len(index_list)))
    # Comparar listas
    if index_list == expected_indices:
        print("Los índices son consecutivos.")
    else:
        print("Los índices no son consecutivos.")
        print("Índices actuales:", index_list)
        print("Índices esperados:", expected_indices)

# Llamar a la función para verificar
chequeo_indices_consecutivos(movies_etl)

Los índices no son consecutivos.
Índices actuales: [30700, 33356, 42222, 43644, 24455, 26564, 26566, 14551, 24351, 23675, 24873, 44274, 26567, 292, 26560, 12481, 536, 17818, 43286, 33361, 44271, 24121, 23692, 42902, 42309, 43301, 23673, 2843, 44624, 45202, 26565, 42170, 23753, 39694, 314, 39517, 45014, 42168, 43255, 351, 6390, 45031, 39331, 42214, 26568, 30543, 256, 30690, 522, 17588, 25206, 834, 30401, 5481, 45167, 38850, 2211, 44842, 42185, 4766, 45204, 44956, 26558, 42169, 1176, 24366, 44009, 1178, 41489, 23358, 42853, 40025, 43642, 41142, 43231, 1152, 21941, 41400, 40251, 10309, 36223, 12920, 15408, 21123, 43238, 45139, 42967, 34502, 2458, 27478, 25084, 22879, 1184, 4863, 23465, 41150, 25390, 26555, 24860, 31072, 11827, 41860, 13133, 19129, 44678, 40598, 21878, 25797, 25865, 44925, 14425, 5678, 11008, 42171, 5814, 5215, 26553, 7000, 16974, 42355, 15480, 41492, 26569, 10122, 7725, 45145, 25534, 43221, 17124, 28780, 23748, 11780, 41809, 23737, 28830, 20505, 1639, 26562, 11308, 30396,

Para mantener organizado el dataframe, se procede a reorganizar los índices de las filas del dataframe. Para ello, se utiliza el método 'reset_index()`.

In [43]:
# Reorganizar las filas de forma consecutiva
movies_etl = movies_etl.reset_index(drop=True)

In [44]:
# Se verifica de nuevo los índices si se han reorganizado efectivamente. 
chequeo_indices_consecutivos(movies_etl)

Los índices son consecutivos.


In [45]:
movies_etl

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,211672,Despicable Me Collection,74000000,"[Family, Animation, Adventure, Comedy]",en,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,"[Universal Pictures, Illumination Entertainment]",[United States of America],2015-06-17,1156730962,91,[English],Released,"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015,15.631499
1,297762,Wonder Woman Collection,149000000,"[Action, Adventure, Fantasy]",en,An Amazon princess comes to the world of Man t...,294.337037,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],2017-05-30,820580447,141,"[Deutsch, English]",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017,5.507251
2,321612,,160000000,"[Family, Fantasy, Romance]",en,A live-action adaptation of Disney's version o...,287.253654,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",2017-03-16,1262886337,129,[English],Released,Be our guest.,Beauty and the Beast,6.8,5530,2017,7.893040
3,339403,,34000000,"[Action, Crime]",en,After being coerced into working for a crime b...,228.032744,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",2017-06-28,224511319,113,[English],Released,All you need is one killer track.,Baby Driver,7.2,2083,2017,6.603274
4,177572,,165000000,"[Adventure, Family, Animation, Action, Comedy]",en,The special bond that develops between plus-si...,213.849907,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],2014-10-24,652105443,102,[English],Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014,3.952154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45019,389854,,0,[],en,Directed by,0.000000,[],[],1989-03-25,0,0,[],Released,nan,The Last Ferry,0.0,0,1989,0.000000
45020,317965,,0,[],it,A young shy man (Tommaso) runs down a blond gi...,0.000000,[],[],1993-03-12,0,111,[],Released,nan,La Bionda,0.0,0,1993,0.000000
45021,315256,,0,[Horror],hi,"Dr. Vishal, a mad scientist, turns into a bloo...",0.000000,[Ramsay Productions],[India],1981-06-19,0,0,[हिन्दी],Released,nan,Terror,0.0,0,1981,0.000000
45022,293552,,0,[],en,This moving documentary recounts the two month...,0.000000,[],[],1993-01-01,0,0,[],Released,nan,At the River I Stand,0.0,0,1993,0.000000


# SEGUNDO DATAFRAME: 'credits.csv'

In [46]:
# Se Carga el archivo CSV en un DataFrame
credits = pd.read_csv('..\credits.csv', encoding='utf-8')

In [47]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


Desanidado de la columna 'cast'

In [48]:
# Creamos lista final para nombres del cast
all_cast_names = []

# Recorremos cada fila del Data Frame
for index, row in credits.iterrows():
    
    # Convertimos strings a lista de diccionarios
    try:
        cast_list = ast.literal_eval(str(row['cast']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        cast_list = []

    # Creamos una lista temporal para nombres de cast en la fila actual
    cast_names = []

    # Extraemos los datos de cada diccionario en la lista
    for cast in cast_list:
        cast_names.append(cast.get('name', ''))

    # Añadimos las listas temporales a las listas finales
    all_cast_names.append(cast_names)

# Chequeamos que el resultado sea correcto
all_cast_names

[['Tom Hanks',
  'Tim Allen',
  'Don Rickles',
  'Jim Varney',
  'Wallace Shawn',
  'John Ratzenberger',
  'Annie Potts',
  'John Morris',
  'Erik von Detten',
  'Laurie Metcalf',
  'R. Lee Ermey',
  'Sarah Freeman',
  'Penn Jillette'],
 ['Robin Williams',
  'Jonathan Hyde',
  'Kirsten Dunst',
  'Bradley Pierce',
  'Bonnie Hunt',
  'Bebe Neuwirth',
  'David Alan Grier',
  'Patricia Clarkson',
  'Adam Hann-Byrd',
  'Laura Bell Bundy',
  'James Handy',
  'Gillian Barber',
  'Brandon Obray',
  'Cyrus Thiedeke',
  'Gary Joseph Thorup',
  'Leonard Zola',
  'Lloyd Berry',
  'Malcolm Stewart',
  'Annabel Kershaw',
  'Darryl Henriques',
  'Robyn Driscoll',
  'Peter Bryant',
  'Sarah Gilson',
  'Florica Vlad',
  'June Lion',
  'Brenda Lockmuller'],
 ['Walter Matthau',
  'Jack Lemmon',
  'Ann-Margret',
  'Sophia Loren',
  'Daryl Hannah',
  'Burgess Meredith',
  'Kevin Pollak'],
 ['Whitney Houston',
  'Angela Bassett',
  'Loretta Devine',
  'Lela Rochon',
  'Gregory Hines',
  'Dennis Haysbert',
 

In [49]:
# Añadimos la nueva columna desanidada al dataframe 'credits'
credits['cast'] = all_cast_names

Desanidado de columna 'Crew', quedandonos solo con los directores

In [50]:
# Creamos lista final para nombres de directores
all_director_names = []

# Recorremos cada fila del DataFrame
for index, row in credits.iterrows():
    # Convertimos la celda en string y luego en lista de diccionarios (manejamos los errores para que continue el proceso)
    try:
        crew_list = ast.literal_eval(str(row['crew']))

    # Si se encuentran valores nulos se corrige el error almacenando una lista vacia.
    except (ValueError, SyntaxError) as e:
        print(f"Error en indice {index} al convertir la cadena en lista de diccionarios.\nError {e} (Valor nulo).")
        crew_list = []

    # Creamos lista temporal para el nombre de los director en la fila actual
    director_names = []

    # Extraer el nombre de los directores de cada diccionario en la lista
    for crew_member in crew_list:
        if crew_member.get('job') == 'Director':
            director_name = crew_member.get('name', '')
            director_names.append(director_name)

    # Añadimos el nombre de los directores a la lista final
    all_director_names.append(director_names)

# Chequeamos que el resultado sea correcto
all_director_names

[['John Lasseter'],
 ['Joe Johnston'],
 ['Howard Deutch'],
 ['Forest Whitaker'],
 ['Charles Shyer'],
 ['Michael Mann'],
 ['Sydney Pollack'],
 ['Peter Hewitt'],
 ['Peter Hyams'],
 ['Martin Campbell'],
 ['Rob Reiner'],
 ['Mel Brooks'],
 ['Simon Wells'],
 ['Oliver Stone'],
 ['Renny Harlin'],
 ['Martin Scorsese'],
 ['Ang Lee'],
 ['Allison Anders',
  'Alexandre Rockwell',
  'Robert Rodriguez',
  'Quentin Tarantino'],
 ['Steve Oedekerk'],
 ['Joseph Ruben'],
 ['Barry Sonnenfeld'],
 ['Jon Amiel'],
 ['Richard Donner'],
 ['Victor Salva'],
 ['Mike Figgis'],
 ['Oliver Parker'],
 ['Lesli Linka Glatter'],
 ['Roger Michell'],
 ['Jean-Pierre Jeunet', 'Marc Caro'],
 ['Zhang Yimou'],
 ['John N. Smith'],
 ['Terry Gilliam'],
 ['Jean-Jacques Annaud'],
 ['Chris Noonan'],
 ['Christopher Hampton'],
 ['Tim Robbins'],
 ['Stephen Low'],
 ['Andy Tennant'],
 ['Amy Heckerling'],
 ['Darrell James Roodt'],
 ['Richard Loncraine'],
 ['Albert Hughes', 'Allen Hughes'],
 ['Michael Hoffman'],
 ['Paul W.S. Anderson'],
 ['Gu

In [51]:
# Añadimos la nueva columna desanidada al dataframe 'credits'
credits['crew'] = all_director_names

In [52]:
credits 

,cast,crew,id
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch],15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker],31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer],11862
...,...,...,...
45471,"[Leila Hatami, Kourosh Tahami, Elham Korda]",[Hamid Nematollah],439050
45472,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Lav Diaz],111109
45473,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",[Mark L. Lester],67758
45474,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[Yakov Protazanov],227506


Left Join con el DataFrame 'movies_etl'

In [53]:
#Este código combina dos DataFrames, movies_etl y credits, en un nuevo DataFrame df_final_etl, basándose en la columna común id. El parámetro how='left' significa que se incluirán todas las filas de movies_etl y se agregarán las filas coincidentes de credits si están disponibles.
df_final_etl = pd.merge(movies_etl,credits, on='id', how='left')
df_final_etl

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,211672,Despicable Me Collection,74000000,"[Family, Animation, Adventure, Comedy]",en,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,"[Universal Pictures, Illumination Entertainment]",[United States of America],2015-06-17,...,[English],Released,"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015,15.631499,"[Sandra Bullock, Jon Hamm, Michael Keaton, All...","[Kyle Balda, Pierre Coffin]"
1,297762,Wonder Woman Collection,149000000,"[Action, Adventure, Fantasy]",en,An Amazon princess comes to the world of Man t...,294.337037,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],2017-05-30,...,"[Deutsch, English]",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017,5.507251,"[Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",[Patty Jenkins]
2,321612,,160000000,"[Family, Fantasy, Romance]",en,A live-action adaptation of Disney's version o...,287.253654,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",2017-03-16,...,[English],Released,Be our guest.,Beauty and the Beast,6.8,5530,2017,7.893040,"[Emma Watson, Dan Stevens, Luke Evans, Kevin K...",[Bill Condon]
3,339403,,34000000,"[Action, Crime]",en,After being coerced into working for a crime b...,228.032744,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",2017-06-28,...,[English],Released,All you need is one killer track.,Baby Driver,7.2,2083,2017,6.603274,"[Ansel Elgort, Lily James, Kevin Spacey, Jamie...",[Edgar Wright]
4,177572,,165000000,"[Adventure, Family, Animation, Action, Comedy]",en,The special bond that develops between plus-si...,213.849907,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],2014-10-24,...,[English],Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014,3.952154,"[Scott Adsit, Ryan Potter, Daniel Henney, T.J....","[Chris Williams, Don Hall]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45063,389854,,0,[],en,Directed by,0.000000,[],[],1989-03-25,...,[],Released,nan,The Last Ferry,0.0,0,1989,0.000000,[],[Waldemar Krzystek]
45064,317965,,0,[],it,A young shy man (Tommaso) runs down a blond gi...,0.000000,[],[],1993-03-12,...,[],Released,nan,La Bionda,0.0,0,1993,0.000000,"[Nastassja Kinski, Sergio Rubini]",[Sergio Rubini]
45065,315256,,0,[Horror],hi,"Dr. Vishal, a mad scientist, turns into a bloo...",0.000000,[Ramsay Productions],[India],1981-06-19,...,[हिन्दी],Released,nan,Terror,0.0,0,1981,0.000000,"[Navin Nischol, Sarika]","[Shyam Ramsay, Tulsi Ramsay]"
45066,293552,,0,[],en,This moving documentary recounts the two month...,0.000000,[],[],1993-01-01,...,[],Released,nan,At the River I Stand,0.0,0,1993,0.000000,[],[]


In [54]:
duplicados_id = df_final_etl['id'].duplicated().sum()
print(f"Cantidad de valores duplicados en la columna 'id': {duplicados_id}")

Cantidad de valores duplicados en la columna 'id': 44


In [55]:
filas_duplicadas = df_final_etl[df_final_etl.duplicated(subset='id', keep=False)]
print(filas_duplicadas)

           id belongs_to_collection    budget  \
1627   109962                               0   
1628   109962                               0   
1635   265189                               0   
1636   265189                               0   
2080     4912                        30000000   
...       ...                   ...       ...   
43722  187156                               0   
44126   99080                               0   
44127   99080                               0   
44937  119916                               0   
44938  119916                               0   

                                          genres original_language  \
1627                                     [Drama]                en   
1628                                     [Drama]                en   
1635                             [Comedy, Drama]                sv   
1636                             [Comedy, Drama]                sv   
2080   [Comedy, Crime, Drama, Romance, Thriller]             

In [56]:
# Se establece anteriormente que la columna identificadora del registro es la columna 'id'. Se revisaron los datos y se determinó que tiene
# los mismos datos en todas sus columnas, solo que la columna de director, cuando es más de un individuo, cambia de orden, así que se pueden eliminar
# los duplicados sin un orden o parámetro en específico. 
df_final_etl = df_final_etl.drop_duplicates('id', keep='first')

In [57]:
# Se verifica la cantidad de duplicados después de realizar la eliminación de los duplicados
duplicados_id = df_final_etl['id'].duplicated().sum()
print(f"Cantidad de valores duplicados en la columna 'id': {duplicados_id}")

Cantidad de valores duplicados en la columna 'id': 0


In [58]:
# Esta función calcula la diferencia entre cada índice y el anterior, y verifica si todas las diferencias son iguales a 1.
if (df_final_etl.index.diff() == 1).all():
    print("Los índices son consecutivos")
else:
    print("Los índices no son consecutivos")

Los índices no son consecutivos


In [59]:
# Verificar si los índices son consecutivos
def chequeo_indices_consecutivos(df_final_etl):
    # Obtener los índices como una lista
    index_list = df_final_etl.index.tolist()
    # Generar una lista de índices consecutivos esperados
    expected_indices = list(range(len(index_list)))
    # Comparar listas
    if index_list == expected_indices:
        print("Los índices son consecutivos.")
    else:
        print("Los índices no son consecutivos.")
        print("Índices actuales:", index_list)
        print("Índices esperados:", expected_indices)

# Llamar a la función para verificar
chequeo_indices_consecutivos(df_final_etl)

Los índices no son consecutivos.
Índices actuales: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211

Para mantener organizado el dataframe, se procede a reorganizar los índices de las filas del dataframe. Para ello, se utiliza el método 'reset_index()`.

In [60]:
# Reorganizar las filas de forma consecutiva
df_final_etl = df_final_etl.reset_index(drop=True)

In [61]:
# Se verifica de nuevo los índices si se han reorganizado efectivamente. 
chequeo_indices_consecutivos(df_final_etl)

Los índices son consecutivos.


In [62]:
# Verificar si los índices son consecutivos
def chequeo_indices_consecutivos(df_final_etl):
    # Obtener los índices como una lista
    index_list = df_final_etl.index.tolist()
    # Generar una lista de índices consecutivos esperados
    expected_indices = list(range(len(index_list)))
    # Comparar listas
    if index_list == expected_indices:
        print("Los índices son consecutivos.")
    else:
        print("Los índices no son consecutivos.")
        print("Índices actuales:", index_list)
        print("Índices esperados:", expected_indices)

# Llamar a la función para verificar
chequeo_indices_consecutivos(df_final_etl)

Los índices son consecutivos.


In [63]:
df_final_etl

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,211672,Despicable Me Collection,74000000,"[Family, Animation, Adventure, Comedy]",en,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,"[Universal Pictures, Illumination Entertainment]",[United States of America],2015-06-17,...,[English],Released,"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015,15.631499,"[Sandra Bullock, Jon Hamm, Michael Keaton, All...","[Kyle Balda, Pierre Coffin]"
1,297762,Wonder Woman Collection,149000000,"[Action, Adventure, Fantasy]",en,An Amazon princess comes to the world of Man t...,294.337037,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],2017-05-30,...,"[Deutsch, English]",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017,5.507251,"[Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",[Patty Jenkins]
2,321612,,160000000,"[Family, Fantasy, Romance]",en,A live-action adaptation of Disney's version o...,287.253654,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",2017-03-16,...,[English],Released,Be our guest.,Beauty and the Beast,6.8,5530,2017,7.893040,"[Emma Watson, Dan Stevens, Luke Evans, Kevin K...",[Bill Condon]
3,339403,,34000000,"[Action, Crime]",en,After being coerced into working for a crime b...,228.032744,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",2017-06-28,...,[English],Released,All you need is one killer track.,Baby Driver,7.2,2083,2017,6.603274,"[Ansel Elgort, Lily James, Kevin Spacey, Jamie...",[Edgar Wright]
4,177572,,165000000,"[Adventure, Family, Animation, Action, Comedy]",en,The special bond that develops between plus-si...,213.849907,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],2014-10-24,...,[English],Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014,3.952154,"[Scott Adsit, Ryan Potter, Daniel Henney, T.J....","[Chris Williams, Don Hall]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45019,389854,,0,[],en,Directed by,0.000000,[],[],1989-03-25,...,[],Released,nan,The Last Ferry,0.0,0,1989,0.000000,[],[Waldemar Krzystek]
45020,317965,,0,[],it,A young shy man (Tommaso) runs down a blond gi...,0.000000,[],[],1993-03-12,...,[],Released,nan,La Bionda,0.0,0,1993,0.000000,"[Nastassja Kinski, Sergio Rubini]",[Sergio Rubini]
45021,315256,,0,[Horror],hi,"Dr. Vishal, a mad scientist, turns into a bloo...",0.000000,[Ramsay Productions],[India],1981-06-19,...,[हिन्दी],Released,nan,Terror,0.0,0,1981,0.000000,"[Navin Nischol, Sarika]","[Shyam Ramsay, Tulsi Ramsay]"
45022,293552,,0,[],en,This moving documentary recounts the two month...,0.000000,[],[],1993-01-01,...,[],Released,nan,At the River I Stand,0.0,0,1993,0.000000,[],[]


Nos aseguramos que no se genere un valor faltante al crear la nueva columna

In [64]:
df_final_etl['cast'] = df_final_etl['cast'].apply(lambda x: x if isinstance(x, list) else [])
df_final_etl['crew'] = df_final_etl['crew'].apply(lambda x: x if isinstance(x, list) else [])

In [65]:
df_final_etl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45024 entries, 0 to 45023
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45024 non-null  int64         
 1   belongs_to_collection  45024 non-null  object        
 2   budget                 45024 non-null  int64         
 3   genres                 45024 non-null  object        
 4   original_language      45024 non-null  object        
 5   overview               45024 non-null  object        
 6   popularity             45024 non-null  float64       
 7   production_companies   45024 non-null  object        
 8   production_countries   45024 non-null  object        
 9   release_date           45024 non-null  datetime64[ns]
 10  revenue                45024 non-null  int64         
 11  runtime                45024 non-null  int64         
 12  spoken_languages       45024 non-null  object        
 13  s

In [66]:
''' # Convierte las listas en tuplas en una  específica
df_final_etl['genres'] = df_final_etl['genres'].apply(tuple)
df_final_etl['production_companies'] = df_final_etl['production_companies'].apply(tuple)
df_final_etl['production_countries'] = df_final_etl['production_countries'].apply(tuple)
df_final_etl['spoken_languages'] = df_final_etl['spoken_languages'].apply(tuple)
df_final_etl['cast'] = df_final_etl['cast'].apply(tuple)
df_final_etl['crew'] = df_final_etl['crew'].apply(tuple)'''


" # Convierte las listas en tuplas en una  específica\ndf_final_etl['genres'] = df_final_etl['genres'].apply(tuple)\ndf_final_etl['production_companies'] = df_final_etl['production_companies'].apply(tuple)\ndf_final_etl['production_countries'] = df_final_etl['production_countries'].apply(tuple)\ndf_final_etl['spoken_languages'] = df_final_etl['spoken_languages'].apply(tuple)\ndf_final_etl['cast'] = df_final_etl['cast'].apply(tuple)\ndf_final_etl['crew'] = df_final_etl['crew'].apply(tuple)"

In [67]:
df_final_etl

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,211672,Despicable Me Collection,74000000,"[Family, Animation, Adventure, Comedy]",en,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,"[Universal Pictures, Illumination Entertainment]",[United States of America],2015-06-17,...,[English],Released,"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015,15.631499,"[Sandra Bullock, Jon Hamm, Michael Keaton, All...","[Kyle Balda, Pierre Coffin]"
1,297762,Wonder Woman Collection,149000000,"[Action, Adventure, Fantasy]",en,An Amazon princess comes to the world of Man t...,294.337037,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],2017-05-30,...,"[Deutsch, English]",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017,5.507251,"[Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",[Patty Jenkins]
2,321612,,160000000,"[Family, Fantasy, Romance]",en,A live-action adaptation of Disney's version o...,287.253654,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",2017-03-16,...,[English],Released,Be our guest.,Beauty and the Beast,6.8,5530,2017,7.893040,"[Emma Watson, Dan Stevens, Luke Evans, Kevin K...",[Bill Condon]
3,339403,,34000000,"[Action, Crime]",en,After being coerced into working for a crime b...,228.032744,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",2017-06-28,...,[English],Released,All you need is one killer track.,Baby Driver,7.2,2083,2017,6.603274,"[Ansel Elgort, Lily James, Kevin Spacey, Jamie...",[Edgar Wright]
4,177572,,165000000,"[Adventure, Family, Animation, Action, Comedy]",en,The special bond that develops between plus-si...,213.849907,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],2014-10-24,...,[English],Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014,3.952154,"[Scott Adsit, Ryan Potter, Daniel Henney, T.J....","[Chris Williams, Don Hall]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45019,389854,,0,[],en,Directed by,0.000000,[],[],1989-03-25,...,[],Released,nan,The Last Ferry,0.0,0,1989,0.000000,[],[Waldemar Krzystek]
45020,317965,,0,[],it,A young shy man (Tommaso) runs down a blond gi...,0.000000,[],[],1993-03-12,...,[],Released,nan,La Bionda,0.0,0,1993,0.000000,"[Nastassja Kinski, Sergio Rubini]",[Sergio Rubini]
45021,315256,,0,[Horror],hi,"Dr. Vishal, a mad scientist, turns into a bloo...",0.000000,[Ramsay Productions],[India],1981-06-19,...,[हिन्दी],Released,nan,Terror,0.0,0,1981,0.000000,"[Navin Nischol, Sarika]","[Shyam Ramsay, Tulsi Ramsay]"
45022,293552,,0,[],en,This moving documentary recounts the two month...,0.000000,[],[],1993-01-01,...,[],Released,nan,At the River I Stand,0.0,0,1993,0.000000,[],[]


In [68]:
duplicados_columna= df_final_etl.duplicated(subset=['id']).sum()

In [69]:
duplicados_columna

np.int64(0)

In [70]:
# Se encontraron valores repetidos en el Data Frame, identificamos las filas a eliminar en base al titulo

# Almacenamos la cantidad de filas antes de la limpieza
numPreDrop = df_final_etl.shape[0]

# Limpiamos valores duplicados
df_final_etl.drop_duplicates(subset=['id'], keep='first', inplace=True)

# Almacenamos la cantidad de filas luego de la limpieza y claculamos la diferencia
numAftDrop = df_final_etl.shape[0]
numDroped = numPreDrop - numAftDrop
print(f'Numero de registros antes de la limpieza:{numPreDrop} \nNumero de registros luego de la limpieza:{numAftDrop} \nSe eliminaron en total {numDroped} registros')

Numero de registros antes de la limpieza:45024 
Numero de registros luego de la limpieza:45024 
Se eliminaron en total 0 registros


In [71]:
df_final_etl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45024 entries, 0 to 45023
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45024 non-null  int64         
 1   belongs_to_collection  45024 non-null  object        
 2   budget                 45024 non-null  int64         
 3   genres                 45024 non-null  object        
 4   original_language      45024 non-null  object        
 5   overview               45024 non-null  object        
 6   popularity             45024 non-null  float64       
 7   production_companies   45024 non-null  object        
 8   production_countries   45024 non-null  object        
 9   release_date           45024 non-null  datetime64[ns]
 10  revenue                45024 non-null  int64         
 11  runtime                45024 non-null  int64         
 12  spoken_languages       45024 non-null  object        
 13  s

In [72]:
df_final_etl.isnull().sum()

id                       0
belongs_to_collection    0
budget                   0
genres                   0
original_language        0
overview                 0
popularity               0
production_companies     0
production_countries     0
release_date             0
revenue                  0
runtime                  0
spoken_languages         0
status                   0
tagline                  0
title                    0
vote_average             0
vote_count               0
release_year             0
return                   0
cast                     0
crew                     0
dtype: int64

In [73]:
df_final_etl.describe()

,id,budget,popularity,release_date,revenue,runtime,vote_average,vote_count,release_year,return
count,45024.000000,4.502400e+04,45024.000000,45024,4.502400e+04,45024.000000,45024.000000,45024.000000,45024.000000,4.502400e+04
mean,107271.618626,4.262330e+06,2.945547,1992-05-01 01:40:12.153518080,1.131391e+07,94.218017,5.637178,110.901408,1991.840974,6.652027e+02
min,2.000000,0.000000e+00,0.000000,1874-12-09 00:00:00,0.000000e+00,0.000000,0.000000,0.000000,1874.000000,0.000000e+00
25%,26244.750000,0.000000e+00,0.399384,1978-09-13 00:00:00,0.000000e+00,85.000000,5.000000,3.000000,1978.000000,0.000000e+00
50%,59240.000000,0.000000e+00,1.142413,2001-08-17 00:00:00,0.000000e+00,95.000000,6.000000,10.000000,2001.000000,0.000000e+00
75%,154096.000000,0.000000e+00,3.733815,2010-12-14 06:00:00,0.000000e+00,107.000000,6.800000,35.000000,2010.000000,0.000000e+00
max,469172.000000,3.800000e+08,547.488298,2020-12-16 00:00:00,2.787965e+09,1256.000000,10.000000,14075.000000,2020.000000,1.239638e+07
std,111680.980753,1.750239e+07,6.027979,NaN,6.463279e+07,38.322403,1.897277,493.570967,24.083754,7.498469e+04


In [74]:
df_final_etl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45024 entries, 0 to 45023
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45024 non-null  int64         
 1   belongs_to_collection  45024 non-null  object        
 2   budget                 45024 non-null  int64         
 3   genres                 45024 non-null  object        
 4   original_language      45024 non-null  object        
 5   overview               45024 non-null  object        
 6   popularity             45024 non-null  float64       
 7   production_companies   45024 non-null  object        
 8   production_countries   45024 non-null  object        
 9   release_date           45024 non-null  datetime64[ns]
 10  revenue                45024 non-null  int64         
 11  runtime                45024 non-null  int64         
 12  spoken_languages       45024 non-null  object        
 13  s

# Exportación de archivo en formato .parquet

In [75]:
parquet_filename = 'streaming.parquet'
table = pa.Table.from_pandas(df_final_etl)
pq.write_table(table, parquet_filename)

In [76]:
df_final_etl

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,211672,Despicable Me Collection,74000000,"[Family, Animation, Adventure, Comedy]",en,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,"[Universal Pictures, Illumination Entertainment]",[United States of America],2015-06-17,...,[English],Released,"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015,15.631499,"[Sandra Bullock, Jon Hamm, Michael Keaton, All...","[Kyle Balda, Pierre Coffin]"
1,297762,Wonder Woman Collection,149000000,"[Action, Adventure, Fantasy]",en,An Amazon princess comes to the world of Man t...,294.337037,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],2017-05-30,...,"[Deutsch, English]",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017,5.507251,"[Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",[Patty Jenkins]
2,321612,,160000000,"[Family, Fantasy, Romance]",en,A live-action adaptation of Disney's version o...,287.253654,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",2017-03-16,...,[English],Released,Be our guest.,Beauty and the Beast,6.8,5530,2017,7.893040,"[Emma Watson, Dan Stevens, Luke Evans, Kevin K...",[Bill Condon]
3,339403,,34000000,"[Action, Crime]",en,After being coerced into working for a crime b...,228.032744,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",2017-06-28,...,[English],Released,All you need is one killer track.,Baby Driver,7.2,2083,2017,6.603274,"[Ansel Elgort, Lily James, Kevin Spacey, Jamie...",[Edgar Wright]
4,177572,,165000000,"[Adventure, Family, Animation, Action, Comedy]",en,The special bond that develops between plus-si...,213.849907,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],2014-10-24,...,[English],Released,From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014,3.952154,"[Scott Adsit, Ryan Potter, Daniel Henney, T.J....","[Chris Williams, Don Hall]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45019,389854,,0,[],en,Directed by,0.000000,[],[],1989-03-25,...,[],Released,nan,The Last Ferry,0.0,0,1989,0.000000,[],[Waldemar Krzystek]
45020,317965,,0,[],it,A young shy man (Tommaso) runs down a blond gi...,0.000000,[],[],1993-03-12,...,[],Released,nan,La Bionda,0.0,0,1993,0.000000,"[Nastassja Kinski, Sergio Rubini]",[Sergio Rubini]
45021,315256,,0,[Horror],hi,"Dr. Vishal, a mad scientist, turns into a bloo...",0.000000,[Ramsay Productions],[India],1981-06-19,...,[हिन्दी],Released,nan,Terror,0.0,0,1981,0.000000,"[Navin Nischol, Sarika]","[Shyam Ramsay, Tulsi Ramsay]"
45022,293552,,0,[],en,This moving documentary recounts the two month...,0.000000,[],[],1993-01-01,...,[],Released,nan,At the River I Stand,0.0,0,1993,0.000000,[],[]


In [77]:
df_final_etl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45024 entries, 0 to 45023
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45024 non-null  int64         
 1   belongs_to_collection  45024 non-null  object        
 2   budget                 45024 non-null  int64         
 3   genres                 45024 non-null  object        
 4   original_language      45024 non-null  object        
 5   overview               45024 non-null  object        
 6   popularity             45024 non-null  float64       
 7   production_companies   45024 non-null  object        
 8   production_countries   45024 non-null  object        
 9   release_date           45024 non-null  datetime64[ns]
 10  revenue                45024 non-null  int64         
 11  runtime                45024 non-null  int64         
 12  spoken_languages       45024 non-null  object        
 13  s

In [78]:
chequeo_indices_consecutivos(df_final_etl)

Los índices son consecutivos.
